<h1><center>Laboratorio 6: Optimización de modelos 🧪</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Otoño 2025</strong></center>

### Cuerpo Docente:

- Profesores: Stefano Schiappacasse, Sebastián Tinoco
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Angelo Muñoz, Valentina Zúñiga

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Felipe Hernández
- Nombre de alumno 2: Brandon Peña


### **Link de repositorio de GitHub:** [Repositorio](https://github.com/brandonHaipas/MDS7202-Lab-Prog-Ciencia-de-Datos)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.


### Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: 6 días de plazo con descuento de 1 punto por día. Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda fuertemente asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [ ]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [ ]:
import pickle

from sklearn import set_config
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

from xgboost import XGBRegressor

set_config(transform_output="pandas")

# 1. Separación de conjuntos
random_seed = 200
X = df.drop(columns=['quantity'])
y = df['quantity']

X_train, X_test_val, y_train, y_test_val = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=random_seed,
)

X_val, X_test, y_val, y_test = train_test_split(
    X_test_val,
    y_test_val,
    test_size=1/3,
    random_state=random_seed,
)

# 2. Implementación de FunctionTransformer
def date_parser(df):
    date_labels = ['day', 'month', 'year']
    date_col = df['date']
    df[date_labels] = df['date'].str.split('/', expand=True).astype('category')
    return df

date_transformer = FunctionTransformer(date_parser)

# 3. Implementacion de column transformer
column_transformer = ColumnTransformer(
    [
        (
            "Scale",
            MinMaxScaler(),
            ["lat", "long", "pop", "price"],
        ),
        (
            "OneHot",
            OneHotEncoder(sparse_output=False, handle_unknown="ignore"),
            ["city", "shop", "brand", "container", "capacity", "day", "month", "year"],
        )
    ]
)

column_transformer.set_output(transform='pandas')

# 4. Guardar pasos en un pipeline
dummy_pipeline = Pipeline(
    steps=[
        ("DateTransformer", date_transformer),
        ("Preprocessing", column_transformer),
        ("Regression", DummyRegressor())
    ]
)

# 5. Entrenamiento
dummy_pipeline.fit(X_train, y_train)
dummy_pred = dummy_pipeline.predict(X_val)
print('MAE DummyRegressor:', mean_absolute_error(y_val, dummy_pred))

# 6. Pipeline con XGBRegressor
xgb_pipeline = Pipeline(
    steps=[
        ("DateTransformer", date_transformer),
        ("Preprocessing", column_transformer),
        ("Regression", XGBRegressor(seed=random_seed))
    ]
)

xgb_pipeline.fit(X_train, y_train)
xgb_pred = xgb_pipeline.predict(X_val)
print('MAE XGBRegressor:', mean_absolute_error(y_val, xgb_pred))

# 7. Guardado de modelos en .pkl
with open('dummy.pkl','wb') as f:
    pickle.dump(dummy_pipeline, f)

with open('xgb.pkl','wb') as f:
    pickle.dump(xgb_pipeline, f)

MAE DummyRegressor: 13373.702326110973
MAE XGBRegressor: 2518.02587890625


5. La métrica MAE indica cuando, en promedio, se desvía la predicción del valor real en valor absoluto. En términos de negocio, MAE indica cuánto se desvía la predicción de demanda en promedio de la demanda real, en términos de cantidad. Esta desviación puede ser una sobreestimación o subestimación de la demanda, dado que es en valor absoluto.

6. Con XGBoost la métrica mejora bastante, dado a que se reduce a 2518.03, es decir, un 18.8% del error reportado con el baseline. Luego, es mejor que `DummyRegressor`.

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [ ]:
# 1-2. Pipeline con relación monótona negativa
xgb_neg_pipeline = Pipeline(
    steps=[
        ("DateTransformer", date_transformer),
        ("Preprocessing", column_transformer),
        ("Regression", XGBRegressor(seed=random_seed, monotone_constraints={"Scale__price": -1}))
    ]
)

xgb_neg_pipeline.fit(X_train, y_train)
xgb_neg_pred = xgb_neg_pipeline.predict(X_val)
print('MAE XGBRegressor con restricción monótona negativa:', mean_absolute_error(y_val, xgb_neg_pred))

# 4. Guardado en .pkl
with open('xgb-neg.pkl','wb') as f:
    pickle.dump(xgb_neg_pipeline, f)

MAE XGBRegressor con restricción monótona negativa: 2474.83056640625


3. Se reduce levemente, de 2518.08 a 2474.83. Luego, esta restricción monótona negativa planteada por el amigo parece ser real.

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [ ]:
import optuna
from optuna.samplers import TPESampler

# 1. Definición de objective
def objective(trial):

    # Hiperparametros a optimizar
    xgb_params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "max_leaves": trial.suggest_int("max_leaves", 0, 100),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1),
    }

    one_hot_params = {
        "min_frequency": trial.suggest_float("min_frequency", 0.0, 0.1)
    }

    # Definición de pipeline
    transformer = ColumnTransformer(
        [
            (
                "Scale",
                MinMaxScaler(),
                ["lat", "long", "pop", "price"],
            ),
            (
                "OneHot",
                OneHotEncoder(sparse_output=False, handle_unknown="ignore", min_frequency=one_hot_params["min_frequency"]),
                ["city", "shop", "brand", "container", "capacity", "day", "month", "year"],
            )
        ]
    )

    pipeline = Pipeline(
        steps=[
            ("DateTransformer", date_transformer),
            ("Preprocessing", transformer),
            ("Regression", XGBRegressor(monotone_constraints={"Scale__price": -1}, seed=random_seed, **xgb_params))
        ]
    )

    # Evaluación
    pipeline.fit(X_train, y_train)
    pred = pipeline.predict(X_val)
    mae = mean_absolute_error(y_val, pred)

    # Guardado de pipeline en atributos del trial
    trial.set_user_attr("pipeline", pipeline)

    return mae

# 2-3. Optimización
study = optuna.create_study(direction="minimize")
study.optimize(objective, timeout=300)
print('Número de trials:', len(study.get_trials()))
print('Mejores hiperparámetros:', study.best_trial.params)
print('Mejor MAE:', study.best_value)

# 5. Guardar modelo en pkl
with open('xgb-optuna.pkl','wb') as f:
    pickle.dump(study.best_trial.user_attrs["pipeline"], f)

Número de trials: 181
Mejores hiperparámetros: {'learning_rate': 0.08305700095030882, 'n_estimators': 935, 'max_depth': 8, 'max_leaves': 96, 'min_child_weight': 5, 'reg_alpha': 0.9295078583965931, 'reg_lambda': 0.8362378182907391, 'min_frequency': 0.01861812168395516}
Mejor MAE: 2017.224365234375


3. El MAE reportado con la optimización de hiperparámetros mejora considerablemente respecto a las versiones anteriores, reduciendose a 2017.22 con el mejor conjunto encontrado. Este cambio es esperable, dado que antes se estaban utilizando los hiperparámetros por defecto, pero al ajustar estos al problema el modelo debe funcionar mejor.

4.
    - `learning_rate` es la tasa que pondera los pesos de las nuevas features en cada etapa de boosting, bajo la cual se determina cuanto cada nuevo decision tree a la predicción final. Así, ajusta qué "tan rápido" el modelo aprende. Tenemos un valor de 0.08, por lo que tiene sentido, pensando que el modelo no busca ajustarse rápidamente.
    - `n_estimators` corresponde a la cantidad de decision trees que usa el modelo para la predicción final. El mejor valor encontrado es de 953, muy cercano a la cota superior. Como XGBoost es un método ensamblado, hace sentido que tener una mayor cantidad de árboles sea mejor, y probablemente sobre los 953 el algoritmo comienza a sobreajustarse.
    - `max_depth` corresponde a la profundidad máxima de cada árbol. Siguiendo la lógica anterior, a mayor profundidad, los árboles son más complejos, lo que permite capturar patrones, a su vez, más complejos. Así, se obtiene un valor cercano a la cota superior provista, 8.
    - `max_leaves` es el máximo número de nodos hoja de cada árbol, donde un mayor valor propicia árboles más complejos. Nuevamente se obtiene un valor alto, de 96, lo que se condice con los valores anteriores obtenidos.
    - `min_child_weight` corresponde a la suma mínima de pesos de cada instancia en cada nodo. Mientras más grande es, el algoritmo es más conservador. En este caso, se obtiene el máximo valor provisto, de 5.
    - `reg_alpha` es el valor de regularización L1 en los pesos, y `reg_lambda` es el valor de regularización L2 en los pesos. Mientras mayores son estos valores, la regularización es más agresiva. En ambos casos se obtuvieron valores más cercanos a 1, lo que tiene sentido pues previene que el modelo se sobreajuste a algún peso en particular.
    - `min_frequency`: Especifíca la cantidad mínima (frecuencia) bajo la cual una categoría se considerará infrecuente. Al ser un número flotante entre 0 y 1, se consideran infrecuentes aquellas categorías que se encuentran bajo ese porcentaje respecto al total de samples. En este caso se obtiene un valor de 0.019, lo que implica que aquellas categorías bajo el 1.9% de samples en el conjunto se consideran infrecuentes. El valor hace sentido, pues agrega valores muy extraños en una simple columna tras realizar one-hot encoding.

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [ ]:
!pip install optuna-integration[xgboost]

In [ ]:
import optuna
from optuna.samplers import TPESampler

# Definición de objective con prunning
def objective(trial):

    # Hiperparametros a optimizar
    xgb_params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
        "n_estimators": trial.suggest_int("n_estimators", 50, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "max_leaves": trial.suggest_int("max_leaves", 0, 100),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 5),
        "reg_alpha": trial.suggest_float("reg_alpha", 0, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1),
    }

    one_hot_params = {
        "min_frequency": trial.suggest_float("min_frequency", 0.0, 0.1)
    }

    # Definición de pipeline con prunning
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, observation_key="validation_1-mae")

    transformer = ColumnTransformer(
        [
            (
                "Scale",
                MinMaxScaler(),
                ["lat", "long", "pop", "price"],
            ),
            (
                "OneHot",
                OneHotEncoder(sparse_output=False, handle_unknown="ignore", min_frequency=one_hot_params["min_frequency"]),
                ["city", "shop", "brand", "container", "capacity", "day", "month", "year"],
            )
        ]
    )

    pipeline = Pipeline(
        steps=[
            ("date_transformer", date_transformer),
            ("preprocessing", transformer),
            ("regression", XGBRegressor(monotone_constraints={"Scale__price": -1}, seed=random_seed, **xgb_params, eval_metric="mae", callbacks=[pruning_callback]))
        ]
    )

    # Evaluación
    pipeline[:-1].fit(X_train, y_train)
    X_train_transformed = pipeline[:-1].transform(X_train)
    X_val_transformed = pipeline[:-1].transform(X_val)

    pipeline.fit(
        X_train,
        y_train,
        regression__eval_set=[(X_train_transformed, y_train), (X_val_transformed, y_val)],
        regression__verbose=False
        )
    pred = pipeline.predict(X_val)
    mae = mean_absolute_error(y_val, pred)

    # Guardado de pipeline en atributos del trial
    trial.set_user_attr("pipeline", pipeline)

    return mae

# Optimización
study_prunning = optuna.create_study(direction="minimize")
study_prunning.optimize(objective, timeout=300, show_progress_bar=True)
print('Número de trials:', len(study_prunning.get_trials()))
print('Mejores hiperparámetros:', study_prunning.best_trial.params)
print('Mejor MAE:', study_prunning.best_value)

# Guardar modelo en pkl
with open('xgb-optuna-prunning.pkl','wb') as f:
    pickle.dump(study.best_trial.user_attrs["pipeline"], f)

   0%|          | 00:00/05:00

Número de trials: 82
Mejores hiperparámetros: {'learning_rate': 0.09671815346032624, 'n_estimators': 527, 'max_depth': 10, 'max_leaves': 88, 'min_child_weight': 2, 'reg_alpha': 0.1485167494520978, 'reg_lambda': 0.4683350259316074, 'min_frequency': 0.04950025545361483}
Mejor MAE: 2134.01708984375


1. El pruning consiste en recorrer árboles entrenados de XGBoost, desde las hojas hasta arriba, removiendo nodos en base a una métrica de decisión. Debería impactar el entrenamiento haciendo que este tenga un paso adicional, pero que elimine aquellos nodos que tienden a disminuir el desempeño del modelo final, en este caso, bajo la métrica de MAE.

2. Los resultados son levemente peores que la sección anterior, sin embargo, se alcanzan a realizar menos de la mitad de las trials. Esto tiene sentido, dado que el paso de realizar pruning hace que el entrenamiento tome más tiempo, y se obtiene un MAE levemente peor, pero con menos de la mitad de la exploración. Quizás con árboles con mayor profundidad se podría apreciar mayores efectos de mejoras con pruning. Esto se aprecia en que el mejor hiperparámetro de `max_depth` ahora es de 10, a diferencia de la optimización anterior donde fue de 8.

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [ ]:
# 1. Historial de optimización
from optuna.visualization import plot_optimization_history

plot_optimization_history(study)

In [ ]:
# 2. Coordenadas paralelas
from optuna.visualization import plot_parallel_coordinate

plot_parallel_coordinate(study)

In [ ]:
# 3. Importancia de hiperparámetros
from optuna.visualization import plot_param_importances

plot_param_importances(study)

4. Observando el Optimization History Plot, se observa que los valores comienzan a concentrarse más cerca del best value alrededor de los 90-100 trials.

5. El gráfico de coordenadas paralelas muestra que los mejores valores de MAE obtenidos utilizan:
  - Learning Rates superiores a 0.05.
  - Max depths superiores a 6.
  - Max leaves, generalmente, cerca de 80 en adelante.
  - Min child weight no tiene una tendencia clara, pero la mayoría de los valores más altos pasan por 5.
  - Min frequency inferior a 0.05.
  - Número de estimadores alto, desde 700 en adelante.
  - Regularizaciones L1 en los extremos (más cercanas a 0 o 1), y regularizaciones L2 generalmente superiores, de 0.5 a 0.7.

6. Los hiperparámetros con mayor importancia son `min_frequency` de OneHotEncoder, seguido de `n_estimators` y `learning_rate` de XGBRegressor.

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [ ]:
# 1. Tabla resumen MAE
df_mae = pd.DataFrame({
    'Modelo': ['Baseline', 'XGBoost', 'XGBoost Constraints', 'XGBoost Optuna', 'XGBoost Prunning'],
    'MAE': [mean_absolute_error(y_val, dummy_pred), mean_absolute_error(y_val, xgb_pred), mean_absolute_error(y_val, xgb_neg_pred), study.best_value, study_prunning.best_value]
})

df_mae

,Modelo,MAE
0,Baseline,13373.702326
1,XGBoost,2518.025879
2,XGBoost Constraints,2474.830566
3,XGBoost Optuna,2017.224365
4,XGBoost Prunning,2134.017090


2.  El modelo con mejor rendimiento según la métrica MAE es XGBoost optimizado con Optuna (sin pruning).

In [ ]:
# 3. Prediccion sobre test
with open('xgb-optuna.pkl','rb') as f:
    best_model = pickle.load(f)

    best_pred = best_model.predict(X_test)
    print('MAE con mejor modelo obtenido', mean_absolute_error(y_test, best_pred))

MAE con mejor modelo obtenido 1971.8763427734375


4. Las métricas en el conjunto de testing son mejores que las de validación, en aprocimadamente 46 puntos de MAE. Esto puede deberse a que el conjunto de testing es considerablemente más pequeño (la mitad que el de validación), luego, por aleatoriedad pueden haber mejores predicciones.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://i.pinimg.com/originals/55/3d/42/553d42bea9b10e0662a05aa8726fc7f4.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>